<a href="https://colab.research.google.com/github/kdemertzis/Hydroinformatics/blob/main/06_geoglows_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geoglows work

This is an exercise in simple python skills to get streamflow information and compute some derivative information. Each step will include links to examples or documentation and will contain some pseudo code that you can use to guide you on your solution. You may work together and refer to any internet resources that help solve the problem. This homework assignment covers the following.

1. Figure out the reach id without needing the graphical interface
1. Write your own code to retrieve forecasted and historical streamflow from the geoglows data service
1. Compare your solution to the geoglows package results
1. Generate some plots of the streamflow (plotly or matplotlib)
1. Make plots of streamflow using the geoglows package
1. Use your python skills to compute some derivative information based on the retrieved discharge information.

Write code and add `Text` cells with written responses when necessary.

Deliverables:
1. A pdf printout of the notebook showing all the code and text cells and the outputs of the code cells (e.g. the graphs and any print statements you use). The pdf ensures i can see what you had when you turned it in lest there be problems when i run it. (file -> print in the google colab page).
1. A link to a github gist/google colab notebook copy of this file with your solution code which i can execute. Put this in the comments section of the learningsuite submission or make it a clickable link somewhere easy to find in the pdf.


In [ ]:
!pip install geoglows -q

In [ ]:
import geoglows
import pandas as pd
import requests
import matplotlib
import plotly.graph_objects as go
from io import StringIO
import numpy as np
import scipy
from datetime import *
from datetime import datetime
import datetime as dt

## 1. Find a reach_id
Being required to use the graphical interface to figure out the stream number can be a pain and it gets in the way of having scripts which retrieve and process information for you. Thankfully, i've developed a way around this for you.

1. use the graphical interface to find a stream id, note the lat/lon where you clicked
2. use the geolgows python package to determine the id for that lat/lon
3. do they match? Can you think of reasons why or why not?

You may find these links helpful
- geoglows package documentation for getting streamflow: https://geoglows.readthedocs.io/en/latest/API%20Documentation/streamflow.html 

Reach_ID is 922108 | 
lat: 30.9969,
lon: -102.1417 | 
Using geoglows package indicates the reach is the same

In [ ]:
geoglows.streamflow.latlon_to_reach(30.9969, -102.1417)

## 2. Write code to get data
Write your own code to retrieve the historical simulation and current forecast (ForecastStats). Use the graphical interface to the geoglows model to find the ID of a segment of the Mississipi rive near the outlet to the Gulf of Mexico.

You may find these links helpful
- Documentation for the geoglows rest api: https://geoglows.ecmwf.int/documentation
- The source code for the equivalent functions in the geoglows python package: https://github.com/BYU-Hydroinformatics/geoglows/blob/master/geoglows/streamflow.py 


In [ ]:
# Enter your reach_id here
reach_id = 13082502

In [ ]:
# use the reach id and REST api documentation to build a url to query the geoglows api
restapi_base = 'https://geoglows.ecmwf.int/api/'
url_for_forecast =  restapi_base + 'ForecastStats/?reach_id=' + str(reach_id)
url_for_historical =  restapi_base + 'HistoricSimulation/?reach_id=' + str(reach_id)

In [ ]:
url_for_forecast

In [ ]:
# use requests or some other method to retrieve the csv output
# call your resulting requests object forecast_response and historical_response
# hint: what function did you use in Dr Ames' notebook to download national water model files?
forecast_response = requests.get(url_for_forecast)
historical_response = requests.get(url_for_historical)

In [ ]:
# what you get back from requests is a requests response object.
# it contains some useful information about your request for information from 
# the rest api. run this cell to check it out
forecast_response

In [ ]:
# What we got back from the api is CSV data, but as a long text string. In order to read that into a dataframe in pandas, you will need this code.
forecast_dataframe = pd.read_csv(StringIO(forecast_response.text), index_col=0)
historical_dataframe = pd.read_csv(StringIO(historical_response.text), index_col=0)

## 3. Compare to the geoglows package

Use the following cells (add more if necessary) to retrive the same information from the geoglows python packge. It should take 1 command each for the forecast and historical data. After you've finished, add a cell at the bottom that prints out the dataframes you retrieved using geoglows and using your own code. 

1. Store the results you get from the geoglows package in variables titled geoglows_forecast and geoglows_historical
1. Is the information the same as what you coded by hand?

You may find these links helpful
- geoglows package documentation for getting streamflow: https://geoglows.readthedocs.io/en/latest/API%20Documentation/streamflow.html 

In [ ]:
forecast_stats = geoglows.streamflow.forecast_stats(reach_id=reach_id,return_format='csv',endpoint=restapi_base)
forecast_stats

In [ ]:
historic_simulation = geoglows.streamflow.historic_simulation(reach_id=reach_id, return_format='csv', endpoint=restapi_base)
historic_simulation

## 4. Generate graphs
Use the following cells (add more if necessary) to plot the historical streamflow data you retrieved. You may use plotly, matplotlib, or any other graphing engine you prefer. My personal preference is plotly (and thats what i can help you with if you're stuck).

You may find these links helpful
- https://plotly.com/python/line-and-scatter/ 
- https://plotly.com/python/reference/layout/ 
- https://www.tutorialspoint.com/plotly/plotly_with_pandas_and_cufflinks.htm
- https://www.google.com/search?q=plot+pandas+dataframe 


In [ ]:
# you might want to use this call to experiment with selecting parts of a dataframe
historical_dataframe['streamflow_m^3/s'].values

In [ ]:
# Guide to making your own plotly plot
# each line on the graph is a python object and you combine each of those into a Figure object
# The first cell of this notebook has already imported plotly with: import plotly.graph_objects as go
# you can build what you need with go.Scatter and go.Figure
# here is the pseudo code which you can edit:

# build a scatter line of your dataset, the x values are time and the y are streamflow
# You need to figure out how to select one of the columns in your dataframe
# bonus: figure out how to change the color or style of the line
hist_flow_scatter = go.Scatter(x=historical_dataframe.index, y=historical_dataframe['streamflow_m^3/s'].values, line_color="#00ff00", fill='tozeroy', name='Historical Simulation')

# bonus: find a way to alter the layout of your plot using the layout option
# if you skip this step, this line should say `layout = {}`
layout = go.Layout(title="Historical Simulation Data - '1979-2020'", xaxis_title='Time (year)', yaxis_title='Streamflow (m^3/s)', showlegend=True)

# Add this scatter set to a plotly figure
hist_flow_figure = go.Figure(data = hist_flow_scatter, layout=layout)

In [ ]:
# display the resulting figure
hist_flow_figure.show()

## 5. Compare to the geoglows package

1. Use the functions in the geoglows python package to generate plots.
1. Are the data being plotted the same (ignore styles and labels)? If they are different, why?

You may find these links helpful
- geoglows package documentation for generating plots: https://geoglows.readthedocs.io/en/latest/API%20Documentation/plots.html

In [ ]:
geoglows.plots.forecast_stats(stats=forecast_stats)

In [ ]:
geoglows.plots.historic_simulation(hist=historic_simulation)

## 6. Compute something new
Use your python skills to do something new with the streamflow information you retrieved. You only need to do 1 additional task. Explain what you want to compute/why it would be useful info, compute it, then show the result. Here are some examples of things you might want to try:
- Figure out how to integrate the hydrograph to get the volume going in to a reservoir
- Compute the average or median of the forecast ensembles and compare against the ForecastStats data
- Figure out how to make the plots in matplotlib or another graphing engine
- Figure out how to style the plotly graphs to match the graphs made by the geoglows package
- Make a plot with the forecasted flow (ensembles or stats) and the ForecastRecords on the same plot
- Compute the flow anomoly - (difference between the average forecasted flow and the daily average flow)

In [ ]:
# Create the dataframe for the ForecastsEnsemble
url_for_Ensembles = restapi_base + 'ForecastEnsembles/?reach_id=' + str(reach_id)
Ensembles_response = requests.get(url_for_Ensembles)
Ensembles_dataframe = pd.read_csv(StringIO(Ensembles_response.text), index_col=0)
# Create a new column that calculates the median value of each ensemble
# This can be done for the mean but upon inspection, the mean for the Ensembles is the same as the mean for the Stats
Ensembles_dataframe['median'] = Ensembles_dataframe.median(axis=1)

In [ ]:
Ensembles_dataframe

In [ ]:
# Plot the Ensembles Scatterplot against the Stats Scatterplot
Ens_flow_scatter = go.Scatter(x=Ensembles_dataframe.index, y=Ensembles_dataframe['median'].values, name='Ensemble Median')

stat_flow_scatter = go.Scatter(x=forecast_dataframe.index, y=forecast_dataframe['flow_avg_m^3/s'].values, name='Stats Average')

layout = go.Layout(title="ForecastStats Average vs ForecastsEnsemble Median", xaxis_title='Time (year)', yaxis_title='Streamflow (m^3/s)', showlegend=True)

fig = go.Figure(layout=layout)

fig.add_trace(Ens_flow_scatter)
fig.add_trace(stat_flow_scatter)

I decided to find out the volumes of the Forecast Stats and Ensemble hydrographs. Maybe there is an easier way to do this but I re-added the index column back to the DataFrame columns, converted it to UTC then to cumulaitve seconds. From there I used the SciPy integration tool to find the total volumes.

In [ ]:
# copy forecast_dataframe to a new df
fcdf = forecast_dataframe[['flow_avg_m^3/s']].copy()
# Remove the null values
fcdf = fcdf.dropna()
# reset the index
fcdf.reset_index(inplace=True)
# convert datetime to UTC and then to seconds
fcdf['datetime'] = pd.to_datetime(fcdf['datetime'], utc=True)
fcdf['Time (sec)'] = fcdf['datetime'].dt.hour * 3600 + \
                     fcdf['datetime'].dt.minute * 60 + \
                     fcdf['datetime'].dt.second
fcdf

In [ ]:
# repeat the same steps for Ensembles_dataframe
endf = Ensembles_dataframe[['median']].copy()
endf = endf.dropna()
endf.reset_index(inplace=True)
endf['datetime'] = pd.to_datetime(endf['datetime'], utc=True)
endf['Time (sec)'] = endf['datetime'].dt.hour * 3600 + \
                     endf['datetime'].dt.minute * 60 + \
                     endf['datetime'].dt.second
endf

In [ ]:
# Calculate the total volumes
fcdf_volume = abs(scipy.integrate.trapz(y=fcdf['flow_avg_m^3/s'].values, x=fcdf['Time (sec)'].values))
endf_volume = abs(scipy.integrate.trapz(y=endf['median'].values, x=endf['Time (sec)'].values))
print("Stats: " + str(fcdf_volume) + ' m^3')
print("Ensemble: " + str(endf_volume) + ' m^3')